In [1]:
import os, sys, warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook as tqdm

import pandas as pd
import numpy as np
from datetime import datetime as dt
import re

import lightgbm as lgb
from catboost import Pool, CatBoostRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from pygam import LinearGAM, s, f

from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedShuffleSplit as SSsplit
from sklearn.metrics import mean_squared_error as mse
def rmse(pred, true) : return np.sqrt(mse(true, pred))

import matplotlib.pyplot as plt
import seaborn as sns

## data preparing

In [2]:
read_path = '../dataset/fin/'
write_path = '../dataset/modelCols/'
train = pd.read_csv(read_path+'train_fe.csv')
test = pd.read_csv(read_path+'test_fe.csv')

In [3]:
files = os.listdir(read_path)
newFiles = []
histFiles = []
transFiles = []
isFile = re.compile(r'.*[.]csv')
isNew = re.compile(r'.*_new[.]csv')
isHist = re.compile(r'.*_hist[.]csv')
isTrans = re.compile(r'.*_trans[.]csv')
for file in tqdm(files):
    if re.match(isFile, file):
        locals()[file[:-4]] = pd.read_csv(read_path+file)
        if re.match(isNew, file): newFiles.append(file[:-4])
        elif re.match(isHist, file): histFiles.append(file[:-4])
        elif re.match(isTrans, file): transFiles.append(file[:-4])
        else : print('is it proper file name? : {}'.format(file))

is it proper file name? : test_fe.csv
is it proper file name? : train_fe.csv



In [4]:
tempCols = ['merchant_try_'+col if col!='card_id' else col for col in locals()['mertry_trans'].columns.tolist()]
locals()['mertry_trans'].columns = tempCols
tempCols = ['merchant_visit_'+col if col!='card_id' else col for col in locals()['mervisit_trans'].columns.tolist()]
locals()['mervisit_trans'].columns = tempCols

In [16]:
train_new = train
for file in tqdm(newFiles):
    train_new = train_new.merge(locals()[file], how='left', on='card_id')
    if train_new.shape[0] != train.shape[0] : raise WrongMergeError

is it proper merge? : (201917, 37) : mer_new


In [5]:
train_hist = train
for file in tqdm(histFiles):
    train_hist = train_hist.merge(locals()[file], how='left', on='card_id')
    if train_hist.shape[0] != train.shape[0] : raise WrongMergeError

In [16]:
sm_feature_train_trans = sm_feature_train_trans[['card_id','Category3_A_rate','Category3_C_rate','pur_mer_sum','pur_mer_mean','trans_merchant']]
sm_feature_test_trans = sm_feature_test_trans[['card_id','Category3_A_rate','Category3_C_rate','pur_mer_sum','pur_mer_mean','trans_merchant']]
sm_feature_train_trans.to_csv('sm_train_trans.csv', index=False)
sm_feature_test_trans.to_csv('sm_test_trans.csv', index=False)

In [24]:
transFiles

['fe01_trans',
 'fe02_trans',
 'fe2_trans',
 'merIdCnvrt_trans',
 'mertry_trans',
 'mervisit_trans',
 'mer_trans',
 'modeKey_trans',
 'purchase_amount_by_ym_city_trans',
 'purchase_amount_by_ym_merchant_trans',
 'purchase_amount_by_ym_trans',
 'regular_FE_trans',
 'sm_test_trans',
 'sm_train_trans',
 'transaction_count_by_ym_trans',
 'trans_count_by_ym_city_trans',
 'trans_count_by_ym_merchant_trans']

In [6]:
train_trans = train
transFiles.remove('sm_test_trans',)
for file in tqdm(transFiles):
    train_trans = train_trans.merge(locals()[file], how='left', on='card_id')
    if train_trans.shape[0] != train.shape[0] : print('it is wrong : {} : {}'.format(train_trans.shape, file))

In [51]:
test_trans = test
transFiles.append('sm_test_trans')
transFiles.remove('sm_train_trans',)

for file in tqdm(transFiles):
    test_trans = test_trans.merge(locals()[file], how='left', on='card_id')
    if test_trans.shape[0] != test.shape[0] : print('it is wrong : {} : {}'.format(test_trans.shape, file))

In [52]:
test_trans[modelCols].to_csv('fin_test.csv', index=False)

In [53]:
train_trans[modelCols].to_csv('fin_train.csv', index=False)

## CV Data

In [7]:
modelCols = train_trans.columns.tolist()
addSkew = ['purchase_day_skew',
'purchase_hour_skew',
'month_lag_skew',
'purchase_date_total_day_skew',
'month_diff_from_trade_skew',
'month_diff_from_today_skew',
'purchase_amount_skew',
'purchase_amount_new_skew',
'purchase_amount_trim_skew',]

rmCols = ['card_id', 'first_active','target', 'outliers', 'purchase_date_max','purchase_date_min']
isOutlier = re.compile(r'.*_outlier')
isSkew = re.compile(r'.*_skew')

catCols = ['first_active_dayofweek', ]
isFeature = re.compile(r'feature_[\d]')
isModeKey = re.compile(r'.*_modeKey')
isTop3Key = re.compile(r'.*_top3_key_mean')

for col in modelCols:
    if re.match(isSkew, col): rmCols.append(col)
    elif re.match(isOutlier, col): rmCols.append(col)
for col in rmCols:
    modelCols.remove(col)
modelCols += addSkew
for col in modelCols:
    if re.match(isFeature, col): catCols.append(col)
    elif re.match(isModeKey, col): catCols.append(col)
#     elif re.match(isTop3Key, col): catCols.append(col)
catCols2 = []
for col in catCols:
    if train_trans[col].isna().sum()==0:
        catCols2.append(modelCols.index(col))

In [8]:
split_rate = test.shape[0]/(train.shape[0]+test.shape[0])
split_y = train['outliers']
SSspliter = SSsplit(3, split_rate)
for i, (train_index, test_index) in enumerate(SSspliter.split(train, split_y)):
    locals()['x_train_'+str(i)] = train_trans.iloc[train_index]
    locals()['x_validate_'+str(i)] = train_trans.iloc[test_index]
    locals()['y_train_'+str(i)] = train_trans['target'].iloc[train_index]
    locals()['y_validate_'+str(i)] = train_trans['target'].iloc[test_index]

In [9]:
modelCols = pd.read_csv(write_path+'trans_401.csv')['modelCols'].values.tolist()
modelCols+=sm_test_trans.columns[1:].tolist()
modelCols+=regular_FE_trans.columns[1:].tolist()

catCols = list(set(modelCols)&set(catCols))
catCols2 = []
for col in catCols:
    if train_trans[col].isna().sum()==0:
        catCols2.append(modelCols.index(col))

## CV

In [10]:

report_lgb = pd.DataFrame(index=modelCols+['RMSE'])
report_xgb = pd.DataFrame(index=modelCols+['RMSE'])
report_cat = pd.DataFrame(index=modelCols+['RMSE'])
for i in tqdm(range(3)):
        x = locals()['x_train_'+str(i)][modelCols]
        y = locals()['y_train_'+str(i)].values
        x_ = locals()['x_validate_'+str(i)][modelCols]
        y_ = locals()['y_validate_'+str(i)].values

        params = {
            'objective':'regression',
            'metric':'l2',
            'num_threads':8,
            'num_iterations': 1000,
        }
        lgb_data = lgb.Dataset(x, label = y)
        bst = lgb.train(params, lgb_data)
        pred = bst.predict(x_)
        score = rmse(pred, y_)
        report = bst.feature_importance().tolist()
        report.append(score)
        report_lgb[i] = report


        model = XGBRegressor(n_estimators=1000,n_jobs=8,)
        model.fit(x, y)
        pred = model.predict(x_)
        score = rmse(pred, y_)
        report = model.feature_importances_.tolist()
        report.append(score)
        report_xgb[i] = report


        train_pool = Pool(x, y, cat_features=catCols2)
        test_pool = Pool(x_, cat_features=catCols2) 

        model = CatBoostRegressor(iterations=1000,
                                  learning_rate=0.01, 
                                  loss_function='RMSE',
                                  thread_count = 8)
        model.fit(train_pool, silent=True)
        pred = model.predict(test_pool)
        score = rmse(pred, y_)
        report = model.get_feature_importance()
        report.append(score)
        report_cat[i] = report

catCols2 = result(0.2)

report_lgb['mean'] = report_lgb.apply(lambda x : x.mean(), axis=1)
report_lgb = report_lgb.sort_values(by='mean', ascending=False)
report_xgb['mean'] = report_xgb.apply(lambda x : x.mean(), axis=1)
report_xgb = report_xgb.sort_values(by='mean', ascending=False)
report_cat['mean'] = report_cat.apply(lambda x : x.mean(), axis=1)
report_cat = report_cat.sort_values(by='mean', ascending=False)

fig = plt.figure(figsize=(100,8))
lgb_ax = plt.subplot(3, 1, 1)
xgb_ax = plt.subplot(3, 1, 2)
cat_ax = plt.subplot(3, 1, 3)
sns.barplot(report_lgb.index, report_lgb['mean'], ax=lgb_ax)
sns.barplot(report_xgb.index, report_xgb['mean'], ax=xgb_ax)
sns.barplot(report_cat.index, report_cat['mean'], ax=cat_ax)
plt.xticks(rotation='vertical')
plt.show()

C:\anaconda3\envs\py37\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\anaconda3\envs\py37\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\anaconda3\envs\py37\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


NameError: name 'result' is not defined

In [12]:
modelCols2 = modelCols

In [17]:
model = modelCols2

In [18]:
catCols2 = result(0.1)

length of drop cols : 40
['purchase_date_total_day_skew', 'trans_count_by_ym_merchant_card_rate_as_ym_var', 'hist_regular_nunique', 'trans_count_diff_by_ym_merchant_card_mean', 'pur_mer_sum', 'trans_count_by_ym_merchant_card_rate_as_merchant_max', 'hist_regular_min', 'trans_count_by_ym_merchant_card_rate_as_ym_mean', 'purchase_hour_skew', 'hist_regular_mode', 'trans_count_by_ym_merchant_card_rate_as_merchant_median', 'trans_count_by_ym_merchant_card_rate_as_ym_quantileRange', 'trans_count_by_ym_merchant_card_rate_as_merchant_var', 'trans_count_by_ym_merchant_card_rate_as_ym_merchant_quantileRange', 'RMSE', 'trans_count_diff_by_ym_merchant_card_min', 'month_diff_from_trade_skew', 'trans_count_by_ym_merchant_card_rate_as_ym_merchant_var', 'Category3_A_rate', 'trans_merchant', 'hist_regular_size', 'trans_count_by_ym_merchant_card_rate_as_ym_merchant_min', 'trans_count_diff_by_ym_merchant_card_var', 'trans_count_by_ym_merchant_card_rate_as_merchant_mean', 'trans_count_by_ym_merchant_card_r

ValueError: list.remove(x): x not in list

## 1차 결과

In [11]:
def result(cut_rate=.1):
    global modelCols, catCols
    per = cut_rate*len(modelCols)
    per = int(per)
    dropCols = set(report_lgb[-per:].index.tolist())&set(report_xgb[-per:].index.tolist())&set(report_cat[-per:].index.tolist())
    dropCols = list(dropCols)
    print('length of drop cols : {}'.format(len(dropCols)), dropCols, sep='\n')
    for col in dropCols: modelCols.remove(col)
    catCols = list(set(modelCols)&set(catCols))
    catCols2 = []
    for col in catCols:
        if train_trans[col].isna().sum()==0:
            catCols2.append(modelCols.index(col))
    print('length of model cols : {}'.format(len(modelCols)))
    print('-'*10+'RMSE'+'-'*10)
    print('''LGB : {}
    XGB : {}
    CAT : {}'''.format(report_lgb['mean']['RMSE'], report_xgb['mean']['RMSE'], report_cat['mean']['RMSE']))
    colDF = pd.DataFrame({'modelCols' : modelCols})
    colDF.to_csv(write_path+'trans_'+str(len(modelCols))+'.csv', index=False)
    print('-'*10+'saved complete'+'-'*10)
    return catCols2

In [11]:
print(len(dropCols), dropCols, sep='\n')

19
{'active_months_lag3_median', 'trans_count_diff_by_ym_merchant_card_quantileRange', 'active_months_lag6_max', 'first_active_elapsed_time_from_today', 'first_active_elapsed_time_from_trade', 'active_months_lag3_max', 'installments_null_cnt', 'purchase_year_min', 'trans_count_diff_by_ym_city_card_quantileRange', 'first_active_quarter', 'first_active_month', 'purchase_amount_diff_by_ym_merchant_card_quantileRange', 'trans_count_diff_by_ym_card_quantileRange', 'active_months_lag6_median', 'first_active_weekofyear', 'first_active_dayofweek', 'first_active_year', 'purchase_amount_diff_by_ym_city_card_quantileRange', 'purchase_amount_diff_by_ym_card_quantileRange'}


## 2차 결과

In [22]:
print(len(dropCols), dropCols, sep='\n')

23
{'days_feature3_trade', 'purchase_year_mode', 'trans_count_by_ym_city_card_min', 'active_months_lag3_quantileRange', 'category_4_new_nunique', 'active_months_lag6_min', 'category_4_new_mode', 'category_1_new_mode', 'purchase_dayofweek_min', 'purchase_amount_over_550', 'trans_count_by_ym_merchant_card_min', 'purchase_weekend_mode', 'days_feature3_trade_ratio', 'active_months_lag12_median', 'merchant_try_min', 'most_recent_purchases_range_new_nunique', 'category_1_new_modeKey', 'installments_new_median', 'feature_min', 'active_months_lag12_max', 'active_months_lag3_min', 'category_2_new_modeKey', 'active_months_lag6_quantileRange'}


In [28]:
len(modelCols)

468

## 3차 결과

In [32]:
print(len(dropCols), dropCols, sep='\n')

11
{'numerical_2_new_max', 'numerical_2_new_min', 'most_recent_purchases_range_new_mode', 'category_5_mode', 'most_recent_sales_range_new_mode', 'category_3_new_modeKey', 'most_recent_sales_range_new_nunique', 'installments_new_min', 'feature_max', 'category_2_new_mode', 'trans_count_by_merchant_ym_min'}


In [33]:
len(modelCols)

457

## 4차 결과

In [37]:
print(len(dropCols), dropCols, sep='\n')

4
{'days_feature2_trade', 'installments_modeKey', 'trans_count_by_ym_city_card_rate_as_city_min', 'purchase_amount_trim_max'}


In [38]:
len(modelCols)

453

## 5차 결과

In [46]:
modelCols = result(modelCols, 0.15)

length of drop cols : 13
['category_5_nunique', 'purchase_hour_max', 'category_2_new_nunique', 'category_3_new_nunique', 'purchase_dayofweek_max', 'category_3_new_mode', 'trans_count_by_ym_merchant_card_rate_as_ym_merchant_max', 'installments_median', 'purchase_month_min', 'state_id_modeKey', 'active_months_lag3_mean', 'feature_mean', 'trans_count_by_ym_card_max']
length of model cols : 440


## 6차 결과

In [59]:
catCols2 = result(0.2)

length of drop cols : 18
['avg_purchases_lag3_max', 'days_feature2_trade_ratio', 'numerical_2_new_sum', 'avg_sales_lag6_max', 'avg_purchases_lag6_max', 'trans_count_by_ym_city_card_median', 'trans_count_by_ym_merchant_card_quantileRange', 'purchase_amount_new_var', 'purchase_day_nunique', 'trans_count_by_ym_merchant_card_median', 'numerical_1_new_min', 'trans_count_diff_by_ym_merchant_card_median', 'city_id_modeKey', 'merchant_visit_mode', 'purchase_amount_new_max', 'purchase_dayofweek_nunique', 'merchant_try_mode', 'purchase_month_max']
length of model cols : 422


NameError: name 'catCol2' is not defined

## 7차 결과

In [ ]:
catCols2 = result(0.2)

## 8차 결과

In [70]:
catCols2 = result(0.2)

length of drop cols : 9
['purchase_dayofweek_mode', 'trans_count_diff_by_ym_card_min', 'trans_count_by_city_ym_rate_median', 'trans_count_by_city_ym_rate_mean', 'trans_count_by_merchant_ym_rate_max', 'trans_count_by_ym_city_card_rate_as_city_var', 'avg_purchases_lag6_var', 'trans_count_by_city_ym_max', 'numerical_2_new_mean']
length of model cols : 401
---saved complete---


## Reg

In [116]:
## 5차 결과

modelCols = result(modelCols, 0.15)report_elastic = pd.DataFrame(index=modelCols)
leng = len(modelCols)
threshold = 1.0e-5
for i in tqdm(range(5)):
    x = locals()['x_train_'+str(i)]
    y = locals()['y_train_'+str(i)].values
    x_ = locals()['x_validate_'+str(i)]
    y_ = locals()['y_validate_'+str(i)].values

    model = ElasticNet()
    while 1 :
        print(threshold)
        model2 = SelectFromModel(model, threshold=threshold)
        model2.fit(x,y)
        temp = model2.get_support().sum()
        if temp > leng*0.95 : threshold *= 10
        elif temp < leng*0.8 : threshold *= 0.1
        else : break
    report_elastic[i] = model2.get_support()
report_elastic['mean'] = report_elastic.apply(lambda x : x.mean(), axis=1)

1e-05
1.0000000000000002e-06
1.0000000000000002e-07
1.0000000000000004e-08
1.0000000000000005e-09
1.0000000000000006e-10
1.0000000000000006e-11
1.0000000000000006e-12
1.0000000000000007e-13
1.0000000000000008e-14
1.0000000000000009e-15
1.000000000000001e-16
1.000000000000001e-17
1.000000000000001e-18
1.000000000000001e-19
1.0000000000000011e-20
1.0000000000000012e-21
1.0000000000000012e-22
1.0000000000000013e-23
1.0000000000000014e-24
1.0000000000000014e-25
1.0000000000000015e-26
1.0000000000000015e-27
1.0000000000000015e-28
1.0000000000000016e-29
1.0000000000000017e-30
1.0000000000000016e-31
1.0000000000000017e-32
1.0000000000000018e-33
1.0000000000000019e-34
1.0000000000000019e-35
1.000000000000002e-36
1.000000000000002e-37
1.000000000000002e-38
1.000000000000002e-39
1.0000000000000022e-40
1.0000000000000022e-41
1.0000000000000023e-42
1.0000000000000023e-43
1.0000000000000023e-44
1.0000000000000023e-45
1.0000000000000024e-46
1.0000000000000024e-47
1.0000000000000024e-48
1.00000000000

KeyboardInterrupt: 

In [109]:
report_elastic = pd.DataFrame(index=modelCols)
leng = len(modelCols)
threshold = 1.0e-5
for i in tqdm(range(5)):
    x = locals()['x_train_'+str(i)]
    y = locals()['y_train_'+str(i)].values
    x_ = locals()['x_validate_'+str(i)]
    y_ = locals()['y_validate_'+str(i)].values

    model = Lasso()
    while 1 :
        model2 = SelectFromModel(model, threshold=threshold)
        model2.fit(x,y)
        temp = model2.get_support().sum()
        if temp > leng*0.95 : threshold *= 10
        elif temp < leng*0.85 : threshold *= 0.1
        else : break
    report_elastic[i] = model2.get_support()
report_elastic['mean'] = report_elastic.apply(lambda x : x.mean(), axis=1)

3

In [ ]:
report_elastic = pd.DataFrame(index=modelCols)
leng = len(modelCols)
threshold = 1.0e-5
for i in tqdm(range(5)):
    x = locals()['x_train_'+str(i)]
    y = locals()['y_train_'+str(i)].values
    x_ = locals()['x_validate_'+str(i)]
    y_ = locals()['y_validate_'+str(i)].values

    model = Ridge()
    while 1 :
        model2 = SelectFromModel(model, threshold=threshold)
        model2.fit(x,y)
        temp = model2.get_support().sum()
        if temp > leng*0.95 : threshold *= 10
        elif temp < leng*0.85 : threshold *= 0.1
        else : break
    report_elastic[i] = model2.get_support()
report_elastic['mean'] = report_elastic.apply(lambda x : x.mean(), axis=1)